In [1]:
import pandas as pd
import folium

In [2]:
df = pd.read_csv('/Users/kgidwani/Documents/Purdue/Current/BIOL_597/Projects/4DG/APRDRG_HOSP_latlong.csv')
hosp_ID = pd.read_csv('/Users/kgidwani/Documents/Purdue/Current/BIOL_597/Projects/4DG/HOSPITAL_ID_2019.csv')
df_hosptlatlong = pd.read_csv('/Users/kgidwani/Documents/Purdue/Current/BIOL_597/Projects/4DG/HospLatlong.csv')

## formatting
df_hosptlatlong.columns = ["Latitude", "Longitude", "Hospital_Name"]

# combine dataframes to get relevant data
df_Hosp_ID_latlong = pd.merge(hosp_ID, df_hosptlatlong,
                        on='Hospital_Name',
                        how='inner')
df_Hosp_ID_latlong = df_Hosp_ID_latlong.rename(columns={"Hospital_ID": "HOSPITAL_ID"})

In [3]:
df.drop(columns=["PAYOR1", "SEVERITY", "PATS", "PWC", "TD", "TC", "XC", "XD"], index = 1, inplace=True)

In [4]:
ordered_diag = df.groupby(["HOSPITAL_ID", "APRDRG_DESCRIPTION", "Hospital_Name", "Latitude", "Longitude"])["APRDRG"].count().to_frame(name='count').reset_index()
# print(ordered_diag)
ordered_diag2 = ordered_diag.sort_values(["HOSPITAL_ID", "count"], ascending = (True, False))
# ordered_diag2.to_csv("ordered_diag2.csv", index=False, encoding='utf-8-sig')
# Code for showing top five diag by hospital
topfive_diag2 = ordered_diag2.groupby('HOSPITAL_ID').head(5)

In [5]:
print(topfive_diag2)

       HOSPITAL_ID                                 APRDRG_DESCRIPTION  \
128              2                                    OTHER PNEUMONIA   
47               2                                      HEART FAILURE   
26               2              CHRONIC OBSTRUCTIVE PULMONARY DISEASE   
30               2              CVA & PRECEREBRAL OCCLUSION W INFARCT   
37               2            DISORDERS OF PANCREAS EXCEPT MALIGNANCY   
...            ...                                                ...   
21179          724  DORSAL & LUMBAR FUSION PROC EXCEPT FOR CURVATU...   
21184          724                              HIP JOINT REPLACEMENT   
21173          724  CERVICAL SPINAL FUSION & OTHER BACK/NECK PROC ...   
21197          724                                    OTHER PNEUMONIA   
21172          724                 CELLULITIS & OTHER SKIN INFECTIONS   

                             Hospital_Name   Latitude  Longitude  count  
128              Hancock Regional Hospital  39.79

In [6]:
topfive_diag2.to_csv("top5_diag2.csv", index=False, encoding='utf-8-sig')

In [8]:
concat_ordered_diag2 = pd.read_csv('/Users/kgidwani/Documents/Purdue/Current/BIOL_597/Projects/4DG/top5_diag2.csv')


In [9]:
concat_ordered_diag2['Combo'] = concat_ordered_diag2.agg('{0[APRDRG_DESCRIPTION]} : {0[count]}'.format, axis=1)

In [10]:
concat_ordered_diag2.drop(columns=["APRDRG_DESCRIPTION", "count"], index = 1, inplace=True)


In [11]:
array_agg = lambda x: ' , '.join(x.astype(str))
concated_ordered_diag2 = concat_ordered_diag2.groupby(['HOSPITAL_ID', 'Hospital_Name', 'Latitude', 'Longitude'], as_index=False).agg({'Combo': array_agg})

In [12]:
print(concated_ordered_diag2)

     HOSPITAL_ID                               Hospital_Name   Latitude  \
0              2                   Hancock Regional Hospital  39.794485   
1              3              Ascension St. Vincent Anderson  40.095531   
2              4   Cameron Memorial Community Hospital, Inc.  41.634337   
3              5               Ascension St. Vincent Warrick  38.041813   
4              7  Indiana University Health Bedford Hospital  38.859460   
..           ...                                         ...        ...   
134          479           Regency Hospital of Porter County  41.546446   
135          485      Vibra Hospital of Northwestern Indiana  41.444427   
136          489  Lafayette Regional Rehabilitation Hospital  40.404654   
137          493      Central Indiana AMG Specialty Hospital  40.196898   
138          724         Riverview Health Westfield Hospital  40.043512   

     Longitude                                              Combo  
0   -85.768587  OTHER PNEUMONIA

In [13]:
map_indiana = folium.Map(location=[39.790341,-86.163356])
map_indiana

In [14]:
for i in range(0, len(concated_ordered_diag2)):
    folium.Marker(
        location=[concated_ordered_diag2.iloc[i]['Latitude'], 
        concated_ordered_diag2.iloc[i]['Longitude']],
        popup=concated_ordered_diag2.iloc[i]['Combo'],
    ).add_to(map_indiana)

In [16]:
for i in range(0, len(concated_ordered_diag2)):
    folium.Marker(
        location=[concated_ordered_diag2.iloc[i]['Latitude'], 
        concated_ordered_diag2.iloc[i]['Longitude']],
        popup=concated_ordered_diag2.iloc[i]['Combo'],
        tooltip = concated_ordered_diag2.iloc[i]['Hospital_Name']
    ).add_to(map_indiana)

In [17]:
map_indiana

In [18]:
df = pd.read_csv('HOSPITAL_ID_2019.csv')

In [19]:
df = pd.read_csv('/Users/kgidwani/Documents/Purdue/Current/BIOL_597/Projects/4DG/APRDRG_HOSP_latlong.csv')


In [20]:
df1 = pd.read_csv('HOSPITAL_ID_2019.csv')

In [21]:
print(df1)

     Hospital_ID                                      Hospital_Name  \
0              1      Indiana University Health University Hospital   
1              2                          Hancock Regional Hospital   
2              3                     Ascension St. Vincent Anderson   
3              4          Cameron Memorial Community Hospital, Inc.   
4              5                      Ascension St. Vincent Warrick   
..           ...                                                ...   
145          705                      NW Indiana 24/7 ER & HOSPITAL   
146          710      Hendricks Regional Health Brownsburg Hospital   
147          724                Riverview Health Westfield Hospital   
148          725  Riverview Health Emergency Room & Urgent Care ...   
149          726  Riverview Health Emergency Room & Urgent Care ...   

    Hospital_City  
0    Indianapolis  
1      Greenfield  
2        Anderson  
3          Angola  
4       Boonville  
..            ...  
145    

In [22]:
import requests

In [29]:
for row in df1['Hospital_Name']:
    id_value = row[0]
    input = row.replace(' ', '+')
    response = 'https://maps.googleapis.com/maps/api/geocode/json?address=' + input + '&key=' + key
    get_response = requests.get(response)
    json_home_call = get_response.json()
    lat = json_home_call['results'][0]['geometry']['location']['lat']
    lng = json_home_call['results'][0]['geometry']['location']['lng']

In [30]:
import requests
import csv
import numpy as np

In [31]:
    np.savetext('geoloc.csv', (lat, lng, row), delimiter=',')

AttributeError: module 'numpy' has no attribute 'savetext'

In [32]:
    np.savetxt('geoloc.csv', (lat, lng, row), delimiter=',')

TypeError: Mismatch between array dtype ('<U52') and format specifier ('%.18e')

In [1]:
map_indiana

NameError: name 'map_indiana' is not defined

In [2]:
import pandas as pd
import folium

In [3]:
df = pd.read_csv('/Users/kgidwani/Documents/Purdue/Current/BIOL_597/Projects/4DG/APRDRG_HOSP_latlong.csv')
hosp_ID = pd.read_csv('/Users/kgidwani/Documents/Purdue/Current/BIOL_597/Projects/4DG/HOSPITAL_ID_2019.csv')
df_hosptlatlong = pd.read_csv('/Users/kgidwani/Documents/Purdue/Current/BIOL_597/Projects/4DG/HospLatlong.csv')

## formatting
df_hosptlatlong.columns = ["Latitude", "Longitude", "Hospital_Name"]

# combine dataframes to get relevant data
df_Hosp_ID_latlong = pd.merge(hosp_ID, df_hosptlatlong,
                        on='Hospital_Name',
                        how='inner')
df_Hosp_ID_latlong = df_Hosp_ID_latlong.rename(columns={"Hospital_ID": "HOSPITAL_ID"})


# drop all the columns with data we do not need
df.drop(columns=["PAYOR1", "SEVERITY", "PATS", "PWC", "TD", "TC", "XC", "XD"], index = 1, inplace=True)

# manipulating the data frames to get what we want
ordered_diag = df.groupby(["HOSPITAL_ID", "APRDRG_DESCRIPTION", "Hospital_Name", "Latitude", "Longitude"])["APRDRG"].count().to_frame(name='count').reset_index()
ordered_diag2 = ordered_diag.sort_values(["HOSPITAL_ID", "count"], ascending = (True, False))
topfive_diag2 = ordered_diag2.groupby('HOSPITAL_ID').head(5)
topfive_diag2.to_csv("top5_diag2.csv", index=False, encoding='utf-8-sig')


concat_ordered_diag2 = pd.read_csv('/Users/kgidwani/Documents/Purdue/Current/BIOL_597/Projects/4DG/top5_diag2.csv')
concat_ordered_diag2['Combo'] = concat_ordered_diag2.agg('{0[APRDRG_DESCRIPTION]} : {0[count]}'.format, axis=1)
concat_ordered_diag2.drop(columns=["APRDRG_DESCRIPTION", "count"], index = 1, inplace=True)
array_agg = lambda x: ' , '.join(x.astype(str))
concated_ordered_diag2 = concat_ordered_diag2.groupby(['HOSPITAL_ID', 'Hospital_Name', 'Latitude', 'Longitude'], as_index=False).agg({'Combo': array_agg})

In [4]:

map_indiana = folium.Map(location=[39.790341,-86.163356])
map_indiana

for i in range(0, len(concated_ordered_diag2)):
    folium.Marker(
        location=[concated_ordered_diag2.iloc[i]['Latitude'], 
        concated_ordered_diag2.iloc[i]['Longitude']],
        popup=concated_ordered_diag2.iloc[i]['Combo'],
        tooltip = concated_ordered_diag2.iloc[i]['Hospital_Name']
    ).add_to(map_indiana)

In [5]:
map_indiana

In [6]:
map_indiana.save('HealthTrendsIN.html')

In [1]:
import pandas as pd
import folium

In [2]:
df = pd.read_csv('/Users/kgidwani/Documents/Purdue/Current/BIOL_597/Projects/4DG/APRDRG_HOSP_latlong_ethnicity.csv')

In [3]:
print(df)

        HOSPITAL_ID  PAYOR1  APRDRG  SEVERITY  PATS  PWC   TD          TC  \
0                 1       1       1         2     5    5   31   1606511.0   
1                 1       1       1         3    18   18  223   7753425.0   
2                 1       1       1         4    20   20  679  16399361.0   
3                 1       2       1         2     1    1    9    484893.0   
4                 1       2       1         3     9    9   79   3607596.0   
...             ...     ...     ...       ...   ...  ...  ...         ...   
116856           17       2     758         1     4    4   24      8313.0   
116857           17       2     758         2    10   10   65     36138.0   
116858           17       4     758         1     3    3   24     11116.0   
116859           17       6     758         1     3    3   30     13931.0   
116860           17       6     758         2     3    3   22     18546.0   

                 XC     XD  ... Hospital_City  \
0       321302.2000   6.20

In [6]:
print(df)

        HOSPITAL_ID  APRDRG                           APRDRG_DESCRIPTION  \
0                 1       1  LIVER TRANSPLANT &/OR INTESTINAL TRANSPLANT   
2                 1       1  LIVER TRANSPLANT &/OR INTESTINAL TRANSPLANT   
3                 1       1  LIVER TRANSPLANT &/OR INTESTINAL TRANSPLANT   
4                 1       1  LIVER TRANSPLANT &/OR INTESTINAL TRANSPLANT   
5                 1       1  LIVER TRANSPLANT &/OR INTESTINAL TRANSPLANT   
...             ...     ...                                          ...   
116856           17     758                         BEHAVIORAL DISORDERS   
116857           17     758                         BEHAVIORAL DISORDERS   
116858           17     758                         BEHAVIORAL DISORDERS   
116859           17     758                         BEHAVIORAL DISORDERS   
116860           17     758                         BEHAVIORAL DISORDERS   

                                        Hospital_Name Hospital_City  \
0       Indiana 

In [10]:
ordered_diag = df.groupby(["HOSPITAL_ID", "APRDRG_DESCRIPTION", "Hospital_Name", "Lat", "Long", "Percentage, People Who Are White Alone", 
                            "Percentage, Black Or African American Alone", "Percentage, People Who Are American Indian And Alaska Native Alone", 
                            "Percentage, People Who Are Asian Alone", "Percentage, People Who Are Native Hawaiian And Other Pacific Islander Alone", 
                            "Percentage, People Who Are Some Other Race Alone"])["APRDRG"].count().to_frame(name='count').reset_index()


In [11]:
print(ordered_diag)

       HOSPITAL_ID                                 APRDRG_DESCRIPTION  \
0                1                                     ABDOMINAL PAIN   
1                1                    ACUTE ANXIETY & DELIRIUM STATES   
2                1              ACUTE BRONCHITIS AND RELATED SYMPTOMS   
3                1                                ACUTE KIDNEY INJURY   
4                1                                     ACUTE LEUKEMIA   
...            ...                                                ...   
21438          724                             RESPIRATORY MALIGNANCY   
21439          724               SEPTICEMIA & DISSEMINATED INFECTIONS   
21440          724                 SHOULDER & ELBOW JOINT REPLACEMENT   
21441          724  SHOULDER, UPPER ARM & FOREARM PROCEDURES EXCEP...   
21442          724                                 SYNCOPE & COLLAPSE   

                                       Hospital_Name        Lat       Long  \
0      Indiana University Health University H

In [12]:
ordered_diag2 = ordered_diag.sort_values(["HOSPITAL_ID", "count"], ascending = (True, False))
topfive_diag2 = ordered_diag2.groupby('HOSPITAL_ID').head(5)

In [13]:
print(topfive_diag2)

       HOSPITAL_ID                                 APRDRG_DESCRIPTION  \
44               1            DISORDERS OF PANCREAS EXCEPT MALIGNANCY   
107              1                       MAJOR SMALL BOWEL PROCEDURES   
149              1                       OTHER DISORDERS OF THE LIVER   
65               1   HEPATIC COMA & OTHER MAJOR ACUTE LIVER DISORDERS   
147              1                   OTHER DIGESTIVE SYSTEM DIAGNOSES   
...            ...                                                ...   
21411          724  DORSAL & LUMBAR FUSION PROC EXCEPT FOR CURVATU...   
21416          724                              HIP JOINT REPLACEMENT   
21405          724  CERVICAL SPINAL FUSION & OTHER BACK/NECK PROC ...   
21429          724                                    OTHER PNEUMONIA   
21404          724                 CELLULITIS & OTHER SKIN INFECTIONS   

                                       Hospital_Name        Lat       Long  \
44     Indiana University Health University H

In [14]:
topfive_diag2.to_csv("top5_diag2.csv", index=False, encoding='utf-8-sig')

In [16]:
concat_ordered_diag2 = pd.read_csv('/Users/kgidwani/Documents/Purdue/Current/BIOL_597/Projects/4DG/top5_diag2.csv')
concat_ordered_diag2['Combo'] = concat_ordered_diag2.agg('{0[APRDRG_DESCRIPTION]} : {0[count]}'.format, axis=1)
concat_ordered_diag2.drop(columns=["APRDRG_DESCRIPTION", "count"], index = 1, inplace=True)

In [17]:
print(concat_ordered_diag2)

     HOSPITAL_ID                                  Hospital_Name        Lat  \
0              1  Indiana University Health University Hospital  39.774923   
2              1  Indiana University Health University Hospital  39.774923   
3              1  Indiana University Health University Hospital  39.774923   
4              1  Indiana University Health University Hospital  39.774923   
5              2                      Hancock Regional Hospital  39.794485   
..           ...                                            ...        ...   
695          724            Riverview Health Westfield Hospital  40.043512   
696          724            Riverview Health Westfield Hospital  40.043512   
697          724            Riverview Health Westfield Hospital  40.043512   
698          724            Riverview Health Westfield Hospital  40.043512   
699          724            Riverview Health Westfield Hospital  40.043512   

          Long  Percentage, People Who Are White Alone  \
0   -

In [18]:
array_agg = lambda x: ' , '.join(x.astype(str))
concated_ordered_diag2 = concat_ordered_diag2.groupby(['HOSPITAL_ID', 'Hospital_Name', 'Lat', 'Long', "Percentage, People Who Are White Alone", 
                            "Percentage, Black Or African American Alone", "Percentage, People Who Are American Indian And Alaska Native Alone", 
                            "Percentage, People Who Are Asian Alone", "Percentage, People Who Are Native Hawaiian And Other Pacific Islander Alone", 
                            "Percentage, People Who Are Some Other Race Alone"], as_index=False).agg({'Combo': array_agg})

In [19]:
print(concated_ordered_diag2)

     HOSPITAL_ID                                  Hospital_Name        Lat  \
0              1  Indiana University Health University Hospital  39.774923   
1              2                      Hancock Regional Hospital  39.794485   
2              3                 Ascension St. Vincent Anderson  40.095531   
3              4      Cameron Memorial Community Hospital, Inc.  41.634337   
4              5                  Ascension St. Vincent Warrick  38.041813   
..           ...                                            ...        ...   
135          479              Regency Hospital of Porter County  41.546446   
136          485         Vibra Hospital of Northwestern Indiana  41.444427   
137          489     Lafayette Regional Rehabilitation Hospital  40.404654   
138          493         Central Indiana AMG Specialty Hospital  40.196898   
139          724            Riverview Health Westfield Hospital  40.043512   

          Long  Percentage, People Who Are White Alone  \
0   -

In [24]:
def data_html(row):
    i = row
    Percent_white = concated_ordered_diag2['Percentage, People Who Are White Alone'].iloc[i]
    Percent_black = concated_ordered_diag2['Percentage, Black Or African American Alone'].iloc[i]
    Percent_AIAN = concated_ordered_diag2['Percentage, People Who Are American Indian And Alaska Native Alone'].iloc[i]
    Percent_asian = concated_ordered_diag2['Percentage, People Who Are Asian Alone'].iloc[i]
    Percent_nhpi = concated_ordered_diag2['Percentage, People Who Are Native Hawaiian And Other Pacific Islander Alone'].iloc[i]
    Percent_other = concated_ordered_diag2['Percentage, People Who Are Some Other Race Alone'].iloc[i]
    name = concated_ordered_diag2.iloc[i]['Hospital_Name']
    top5aprdrg = concated_ordered_diag2.iloc[i]['Combo']

    left_col_colour = "#2A799C"
    right_col_colour = "#C5DCE7"

    html = """<!DOCTYPE html>
<html>

<head>
<h4 style="margin-bottom:0"; width="300px">{}</h4>""".format(name) + """

</head>
    <table style="height: 126px; width: 300px;">
<tbody>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">Top 5 most common APRDRG Descriptors</span></td>
<td style="width: 200px;background-color: """+ right_col_colour +""";">{}</td>""".format(top5aprdrg) + """
</tr>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">Percentage, People Who Are White Alone</span></td>
<td style="width: 200px;background-color: """+ right_col_colour +""";">{}</td>""".format(Percent_white) + """
</tr>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">Percentage, Black Or African American Alone</span></td>
<td style="width: 200px;background-color: """+ right_col_colour +""";">{}</td>""".format(Percent_black) + """
</tr>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">Percentage, People Who Are American Indian And Alaska Native Alone</span></td>
<td style="width: 200px;background-color: """+ right_col_colour +""";">{}</td>""".format(Percent_AIAN) + """
</tr>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">Percentage, People Who Are Asian Alone</span></td>
<td style="width: 200px;background-color: """+ right_col_colour +""";">{}</td>""".format(Percent_asian) + """
</tr>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">Percentage, People Who Are Native Hawaiian And Other Pacific Islander Alone</span></td>
<td style="width: 200px;background-color: """+ right_col_colour +""";">{}</td>""".format(Percent_nhpi) + """
</tr>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">Percentage, People Who Are Some Other Race Alone</span></td>
<td style="width: 200px;background-color: """+ right_col_colour +""";">{}</td>""".format(Percent_other) + """
</tr>
</tbody>
</table>
</html>
"""
    return html

In [25]:
import branca

In [28]:
for i in range(0, len(concated_ordered_diag2)):
    html = data_html(i)
    popup_table = branca.element.IFrame(html=html, width=400, height=300)
    popup = folium.Popup(popup_table, parse_html=True)
    folium.Marker(
        location=[concated_ordered_diag2.iloc[i]['Lat'], 
        concated_ordered_diag2.iloc[i]['Long']],
        popup=popup,
        tooltip = concated_ordered_diag2.iloc[i]['Hospital_Name']
    ).add_to(map_indiana)

In [29]:
map_indiana

In [1]:
import pandas as pd
import folium
import branca

# this code below is for printing out results when python syntax is too cumbersome
# I run the code in terminal and export the print statements to a .csv
pd.set_option('display.max_rows', None)

## import all the data files
df = pd.read_csv('/Users/kgidwani/Documents/Purdue/Current/BIOL_597/Projects/4DG/APRDRG_HOSP_latlong_ethnicity.csv')
# hosp_ID = pd.read_csv('/Users/kgidwani/Documents/Purdue/Current/BIOL_597/Projects/4DG/HOSPITAL_ID_2019.csv')
# df_hosptlatlong = pd.read_csv('/Users/kgidwani/Documents/Purdue/Current/BIOL_597/Projects/4DG/HospLatlong.csv')


# drop all the columns with data we do not need
df.drop(columns=["PAYOR1", "SEVERITY", "PATS", "PWC", "TD", "TC", "XC", "XD", "Hosptial_name_city"], index = 1, inplace=True)

# manipulating the data frames to get what we want
ordered_diag = df.groupby(["HOSPITAL_ID", "APRDRG_DESCRIPTION", "Hospital_Name", "Lat", "Long", "Percentage, People Who Are White Alone", 
                            "Percentage, Black Or African American Alone", "Percentage, People Who Are American Indian And Alaska Native Alone", 
                            "Percentage, People Who Are Asian Alone", "Percentage, People Who Are Native Hawaiian And Other Pacific Islander Alone", 
                            "Percentage, People Who Are Some Other Race Alone"])["APRDRG"].count().to_frame(name='count').reset_index()
ordered_diag2 = ordered_diag.sort_values(["HOSPITAL_ID", "count"], ascending = (True, False))
topfive_diag2 = ordered_diag2.groupby('HOSPITAL_ID').head(5)
topfive_diag2.to_csv("top5_diag2.csv", index=False, encoding='utf-8-sig')


concat_ordered_diag2 = pd.read_csv('/Users/kgidwani/Documents/Purdue/Current/BIOL_597/Projects/4DG/top5_diag2.csv')
concat_ordered_diag2['Combo'] = concat_ordered_diag2.agg('{0[APRDRG_DESCRIPTION]} : {0[count]}'.format, axis=1)
concat_ordered_diag2.drop(columns=["APRDRG_DESCRIPTION", "count"], index = 1, inplace=True)
array_agg = lambda x: ' , '.join(x.astype(str))
concated_ordered_diag2 = concat_ordered_diag2.groupby(['HOSPITAL_ID', 'Hospital_Name', 'Lat', 'Long', "Percentage, People Who Are White Alone", 
                            "Percentage, Black Or African American Alone", "Percentage, People Who Are American Indian And Alaska Native Alone", 
                            "Percentage, People Who Are Asian Alone", "Percentage, People Who Are Native Hawaiian And Other Pacific Islander Alone", 
                            "Percentage, People Who Are Some Other Race Alone"], as_index=False).agg({'Combo': array_agg})


## now lets create an HTML table that we can input into the Folium Popup
def data_html(row):
    i = row
    Percent_white = concated_ordered_diag2['Percentage, People Who Are White Alone'].iloc[i]
    Percent_black = concated_ordered_diag2['Percentage, Black Or African American Alone'].iloc[i]
    Percent_AIAN = concated_ordered_diag2['Percentage, People Who Are American Indian And Alaska Native Alone'].iloc[i]
    Percent_asian = concated_ordered_diag2['Percentage, People Who Are Asian Alone'].iloc[i]
    Percent_nhpi = concated_ordered_diag2['Percentage, People Who Are Native Hawaiian And Other Pacific Islander Alone'].iloc[i]
    Percent_other = concated_ordered_diag2['Percentage, People Who Are Some Other Race Alone'].iloc[i]
    name = concated_ordered_diag2.iloc[i]['Hospital_Name']
    top5aprdrg = concated_ordered_diag2.iloc[i]['Combo']

    left_col_colour = "#2A799C"
    right_col_colour = "#C5DCE7"

    html = """<!DOCTYPE html>
<html>

<head>
<h4 style="margin-bottom:0"; width="300px">{}</h4>""".format(name) + """

</head>
    <table style="height: 126px; width: 300px;">
<tbody>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">Top 5 most common APRDRG Descriptors</span></td>
<td style="width: 200px;background-color: """+ right_col_colour +""";">{}</td>""".format(top5aprdrg) + """
</tr>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">Percentage, People Who Are White Alone</span></td>
<td style="width: 200px;background-color: """+ right_col_colour +""";">{}</td>""".format(Percent_white) + """
</tr>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">Percentage, Black Or African American Alone</span></td>
<td style="width: 200px;background-color: """+ right_col_colour +""";">{}</td>""".format(Percent_black) + """
</tr>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">Percentage, People Who Are American Indian And Alaska Native Alone</span></td>
<td style="width: 200px;background-color: """+ right_col_colour +""";">{}</td>""".format(Percent_AIAN) + """
</tr>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">Percentage, People Who Are Asian Alone</span></td>
<td style="width: 200px;background-color: """+ right_col_colour +""";">{}</td>""".format(Percent_asian) + """
</tr>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">Percentage, People Who Are Native Hawaiian And Other Pacific Islander Alone</span></td>
<td style="width: 200px;background-color: """+ right_col_colour +""";">{}</td>""".format(Percent_nhpi) + """
</tr>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">Percentage, People Who Are Some Other Race Alone</span></td>
<td style="width: 200px;background-color: """+ right_col_colour +""";">{}</td>""".format(Percent_other) + """
</tr>
</tbody>
</table>
</html>
"""
    return html





map_indiana = folium.Map(location=[39.790341,-86.163356])
map_indiana

for i in range(0, len(concated_ordered_diag2)):
    html = data_html(i)
    popup_table = branca.element.IFrame(html=html, width=400, height=300)
    popup = folium.Popup(popup_table, parse_html=True)
    folium.Marker(
        location=[concated_ordered_diag2.iloc[i]['Lat'], 
        concated_ordered_diag2.iloc[i]['Long']],
        popup=popup,
        tooltip = concated_ordered_diag2.iloc[i]['Hospital_Name']
    ).add_to(map_indiana)

In [2]:
map_indiana

In [3]:
map_indiana.save('HealthTrendsIN.html')